In [ ]:
import pandas as pd
import os,json,sys,pyemd
from os.path import expanduser
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


In [ ]:
DATA_CONVERGE_PROJECT="sd2e-project-43"

data_converge_base = os.path.join(expanduser("~"), 'sd2e-projects', DATA_CONVERGE_PROJECT)
experiment_dir = os.path.realpath(os.path.join(data_converge_base, 'test/batch_20200525'))
experiment_dir_contents = [os.path.realpath(os.path.join(experiment_dir, x)) for x in os.listdir(experiment_dir)]

experiments = [x for x in  experiment_dir_contents 
               if os.path.isdir(x) and "CRISPR-Short-Duration" in x]

experiments


In [ ]:
## Data Helper functions

def get_record(experiment):
    record = json.load(open(os.path.join(experiment, "record.json")))
    return record

def get_record_file(record, file_type="fc_meta"):
    files = record['files']
    files_of_type = [ x for x in files if file_type in x['name']]
    if len(files_of_type) > 0:
        return files_of_type[0]
    else:
        return None

def get_data(experiment, record, file_type):
    fc_raw_file = get_record_file(record, file_type)
    if fc_raw_file:
        data_df = pd.read_csv(os.path.join(experiment, fc_raw_file['name']))
        return data_df
    else:
        return None

def get_bins(df):
    return [float(x.split("_")[1]) for x in df.columns if "bin" in x]

def get_row_values(df,row_name,id_col):
    df_j = df.loc[df[id_col] == row_name]
    df_j = df_j[[x for x in df_j.columns if "bin" in x]]
    return df_j.values[0]
   
    

In [ ]:
## Analysis functions


def emdist(h1, h2, bin_vals):
    '''
    Calculate earth mover's distance between 2 histograms. Histograms are normalized to mass 1.

    :param h1: a 1-D numpy array representing a histogram with the bin values used to make bin_dist
    :param h2: a 1-D numpy array representing a histogram with the bin values used to make bin_dist
    :param bin_vals: representative points in the bins defining the histograms in data
    :return: a scalar value that is the earth mover's distance between normalized h1 and h2
    '''
    bin_dist = np.array([[np.abs(m - n) for m in bin_vals] for n in bin_vals])
    return pyemd.emd(np.asarray(h1)/float(sum(h1)), np.asarray(h2)/float(sum(h2)), bin_dist)


def do_analysis(experiment,datafile,id_col="sample_id",channel_col="channel",channel_val="BL1-A"):
    # datafile is "fc_raw_log10_stats.csv" or "fc_etl_stats.csv"

    ## load dataset from data converge 
    record = get_record(experiment)
    df = get_data(experiment, record, datafile)
    
    if df is None:
        return None
        
    df = df.loc[df[channel_col] == channel_val]

#     ## Truncated for testing
#     df = df.iloc[:5]

    bins = get_bins(df)   
    ids = list(df[id_col].values)   
    res = np.zeros([len(ids),len(ids)])
    
    for j,s in enumerate(ids):
        s_bin_vals = get_row_values(df,s,id_col)
        for k,t in enumerate(ids[j+1:]):
            t_bin_vals = get_row_values(df,t,id_col)
            score = emdist(s_bin_vals, t_bin_vals, bins)
            res[j,j+k+1] = 10**score
            res[j+k+1,j] = 10**score
    df_results = pd.DataFrame(data=res, index=ids, columns=ids)
    return df_results



In [ ]:
## Run Wasserstein analysis on all processed data sets

for experiment in experiments:
    for datafile in ["fc_raw_log10_stats.csv","fc_etl_stats.csv"]:
        experiment_name = experiment.split("/")[-1]
        print(experiment_name)
        print(datafile)
        fname = experiment_name+"_"+datafile.split(".")[0]+"_wasserstein_dists.csv"
        if not os.path.exists(fname):
            df = do_analysis(experiment,datafile)
            df.to_csv(fname)
